1.Create a unigram inverted index

In [12]:
import os
import pandas as pd
import nltk
from nltk import word_tokenize
from bs4 import BeautifulSoup
nltk.download('punkt')
directory = '/content/drive/MyDrive/CSE508_Winter2023_Dataset/'

doc = 0
postings = {}
doc_name_id={}
for filename in os.listdir(directory):
    f = os.path.join(directory, filename)
    doc_name_id[str(doc)]=filename
    # checking if it is a file
    if os.path.isfile(f):
        read_data = open(f, "r")
        contents = read_data.read().strip()
        if doc % 100 == 0:
            print(doc)
        tokens = word_tokenize(contents)
            
        for token in tokens:
            if token in postings:
                if doc not in postings[token]:
                    postings[token].append(doc)
                else:
                    continue
            else:
                postings[token] = [doc]
        doc += 1

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300


In [13]:
print(postings)
print(postings["third"])

{'shock': [0, 1, 5, 6, 18, 34, 41, 49, 50, 54, 57, 59, 65, 66, 67, 72, 75, 76, 90, 91, 98, 100, 106, 110, 115, 118, 126, 127, 130, 131, 132, 139, 150, 154, 163, 182, 184, 189, 192, 206, 207, 210, 211, 212, 227, 229, 232, 241, 248, 249, 264, 268, 275, 290, 291, 294, 305, 309, 319, 345, 349, 351, 354, 373, 378, 380, 383, 385, 389, 396, 401, 403, 422, 426, 433, 440, 446, 450, 453, 461, 462, 463, 474, 484, 488, 495, 499, 508, 533, 538, 544, 547, 551, 561, 575, 583, 587, 589, 594, 598, 600, 605, 607, 611, 623, 642, 644, 651, 681, 682, 685, 686, 718, 727, 731, 743, 759, 760, 765, 767, 774, 778, 780, 789, 790, 791, 800, 803, 805, 821, 826, 833, 836, 843, 856, 857, 868, 869, 876, 877, 879, 884, 892, 895, 897, 912, 913, 919, 921, 928, 930, 934, 939, 948, 953, 956, 962, 965, 969, 982, 995, 1000, 1001, 1015, 1029, 1030, 1034, 1040, 1041, 1044, 1053, 1062, 1088, 1089, 1103, 1114, 1117, 1118, 1122, 1132, 1138, 1150, 1151, 1154, 1157, 1164, 1166, 1169, 1175, 1183, 1199, 1203, 1204, 1219, 1223, 1235,

2. Use Python’s pickle module to save and load the unigram inverted index.

In [14]:
import pickle
# save dictionary to pickle file
with open('my_filename.pickle', 'wb') as file:
    pickle.dump(postings, file, protocol=pickle.HIGHEST_PROTOCOL)
    
# save dictionary to pickle file
with open('doc_name_id.pickle', 'wb') as file:
    pickle.dump(doc_name_id, file, protocol=pickle.HIGHEST_PROTOCOL)

In [15]:
import pickle
import nltk
from nltk.tokenize import word_tokenize
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [16]:
import numpy as np
 
# function to get unique values
 
 
def unique(list1):
    x = np.array(list1)
    print(np.unique(x))

In [17]:
pickle_off = open("my_filename.pickle", 'rb')
postings = pickle.load(pickle_off)

In [18]:
doc_nametoid = open("doc_name_id.pickle", 'rb')
doc_name_to_id = pickle.load(doc_nametoid)

3. Provide support for the following operations:
a. T1 AND T2
b. T1 AND NOT T2
c. T1 OR T2
d. T1 OR NOT T2

In [19]:
def XORY(x, y):
    x, y = sorted(x), sorted(y) #we sort both the x and y before doing computations on them, this way we can do it fast
    _OR = [] 
    comp = 0 
    i, j = 0, 0 
    while i<len(x) and j<len(y):
        if x[i]==y[j]: 
            _OR.append(y[j]) 
            j += 1
            i += 1 
            comp +=1 
        else: 
            comp += 2 
            if x[i]<y[j]: 
                _OR.append(x[i])
                i += 1
            else:
                _OR.append(y[j]) 
                j += 1
    while i<len(x):
        _OR.append(x[i])
        i += 1 
    while j<len(y):
        _OR.append(y[j])
        j += 1
    return _OR, comp

def XANDY(x, y):
    x, y = sorted(x), sorted(y)
    _AND = []
    comp = 0
    i, j = 0, 0
    while i<len(x) and j<len(y):
        if x[i] == y[j]:
            _AND.append(y[j])
            j += 1
            i += 1
            comp += 1
        else:
            comp += 2
            if x[i]<y[j]:
                i+=1
            else:
                j+=1
    return _AND, comp #returning the and array and the number of comparisons we made

# x or not y = U-(y-x)

def XANDNOTY(list1, list2):
    comparisons = 0
    ANDNOTSet = set()
    ptr1 ,ptr2 = 0,0
    len1 = len(list1)
    len2 = len(list2)
    list1.sort()
    list2.sort()
    while ptr1 < len1 and ptr2 < len2:
        if list1[ptr1] < list2[ptr2]:
            comparisons += 1
            ANDNOTSet.add(list1[ptr1])
            ptr1 += 1
        elif list1[ptr1] > list2[ptr2]:
            comparisons += 2
            ptr2 += 1
        else:
            comparisons += 2
            ptr1 += 1
            ptr2 += 1

    ANDNOTSet = list(ANDNOTSet)
    return ANDNOTSet,comparisons

def XORNOTY(list1, list2):
    comparisons = 0
    ORNOTSet = set()
    list1.sort()
    list2.sort()
    ptr1 ,ptr2 = 0,0
    len1 = len(list1)
    len2 = len(list2)
    while ptr1 < len1 and ptr2 < len2:

        if list1[ptr1] == list2[ptr2]:
            comparisons += 1
            if ptr2 == 0:
                for i in range(0, list2[ptr2] + 1):
                    ORNOTSet.add(i)
            else:
                for i in range(list2[ptr2 - 1] + 1, list2[ptr2] + 1):
                    ORNOTSet.add(i)
            ORNOTSet.add(list1[ptr1])
            ptr1 += 1
            ptr2 += 1
        else:
            comparisons += 2
            if list1[ptr1] > list2[ptr2]:
                if ptr2 == 0:
                    for i in range(1, list2[ptr2]):
                        ORNOTSet.add(i)
                else:
                    for i in range(list2[ptr2 - 1] + 1, list2[ptr2]):
                        ORNOTSet.add(i)
                ptr2 += 1
            else:
                ptr1 += 1
    while ptr2 < len2:
        for i in range(list2[ptr2 - 1] + 1, list2[ptr2]):
            ORNOTSet.add(i)
        ptr2 += 1
    for i in range(list2[ptr2 - 1] + 1, 1400):
        ORNOTSet.add(i)
    
    ORNOTSet = list(ORNOTSet)
    return ORNOTSet,comparisons

In [20]:
def get_name_for_doc_id(doc_id):
    if doc_id in doc_name_to_id:
        return doc_name_to_id[doc_id]
    else:
        return ""

In [21]:
def process(input, operations):
    comparisons = 0
    check_order = 0
    if operations.count("OR") == len(operations) or operations.count("AND") == len(operations):
        check_order = 1
    lists = []
    ops = []
    #terms = preprocess(input)
    tokens = word_tokenize(input)
    for term in tokens:
        #print(term)
        #print(postings[term])
        if term in postings:
            lists.append(postings[term])
        else:
            continue
    #CHECK
    """tokens = word_tokenize(operations)
    for term in tokens:
      ops.append(term)"""
    ops=operations.split(',')

    print(ops)   

    
    list1 = lists[0]
    #list2=  lists[1]
    #print(list2)
    #list1= XORY(list1, list2)
    comp = 0
    for i in range(len(ops)):
        if ops[i] == "OR":
            list1,comp = XORY(list1, lists[i + 1])
        elif ops[i] == 'AND':
            list1,comp = XANDY(list1, lists[i + 1])
        elif ops[i] == 'OR NOT':
            list1,comp = XORNOTY(list1, lists[i + 1])
        elif ops[i] == 'AND NOT':
            list1,comp = XANDNOTY(list1, lists[i + 1])

        comparisons += comp
        print(comparisons)
 
    return list1,comparisons

In [22]:
import nltk
import re
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('punkt')
def preprocess(input):
  txt_l = input.lower()
  txt_tk = nltk.word_tokenize(txt_l)
  str_tk = ' '.join(str(token) for token in txt_tk)
  stopwords_ = stopwords.words('english')
  tokens = nltk.word_tokenize(str_tk)
  tokens_without_stopwords = [token for token in tokens if token.lower() not in stopwords_]
  text_without_stopwords = ' '.join(tokens_without_stopwords)
  txt_punc = re.sub(r'[^\w\s]', '', text_without_stopwords)
  txt_b = txt_punc.replace("  ", " ")
  return txt_b


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [23]:
preprocess("I am is    THINg  ing")

'thing ing'

In [24]:
def main():
    T = int(input('Enter no of queries : ').strip())
    if T == 0:
        return
    print('========== Preparing Dataset ==========', end=' ')
    for i in range(T):
        query = (input('\nYou are requested to enter your query : '))
        op_seq = (input('\nYou are requested to enter your operation : '))
        #op_seq = "OR OR"
        query_=preprocess(query)
        #print(query_)
        a,b=process(query_, op_seq)
        #print(a)
        idtoname=[]
        for k in a:
          idtoname.append(doc_name_to_id[str(k)]) 
        list_set = set(idtoname)
        idtoname=list(list_set)
        
        q=[]
        o=[]
        q = query_.split()
        o= op_seq.split(",")
        #print(q)
        #print(o)
        #print(len(q)-1)
        print("Query: ",i+1, " ",end='')
        for j in range(len(q)-1):
          #print(j)
          print(q[j] + " " + o[j] + " ", end='')
        print(q[len(q)-1])
        print("Document ID : ", a)
        print("Document Names (10 max) : ", idtoname[:10])
        print("Number of documents matched: ", len(idtoname))
        print("Minimum Operation : ", b)

if "__main__":
    main()

Enter no of queries : 1
========== Preparing Dataset ========== 
You are requested to enter your query : flat plate third

You are requested to enter your operation : OR,OR NOT
['OR', 'OR NOT']
311
640
Query:  1  flat OR plate OR NOT third
Document ID :  [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 17

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive
